# Workforce Optimization with Erlang-C

## What This Notebook Does
Given call center demand, this notebook finds the **minimum number of agents needed** to meet service level targets.

**Problem:** 317 calls/day, 95 sec handle time → How many agents do I need?  
**Answer:** Use Erlang-C + Optimization to find optimal staffing

## Notebook Structure
1. **Erlang-C Implementation** - Calculate wait times and occupancy
2. **Optimization Functions** - Find minimum agents (2 approaches)
3. **Example** - Test on real data
4. **Batch Processing** - Apply to entire dataset

# ============================================================
# PART 1: ERLANG-C CALCULATOR
# ============================================================
In this section, we implement the core Erlang-C formula and a simple iterative search to find the minimum number of agents.

## Erlang-C Formula

The **Erlang-C formula** calculates the probability that a call must wait in queue (no agent immediately available):

$$
P_w = \frac{\frac{A^N}{N!} \cdot \frac{N}{N - A}}{\sum_{i=0}^{N-1} \frac{A^i}{i!} + \frac{A^N}{N!} \cdot \frac{N}{N - A}}
$$

### Where:
| Symbol | Description |
|--------|-------------|
| $P_w$ | Probability that a call waits in queue |
| $A$ | Traffic intensity (Erlangs) = $\frac{\lambda \cdot AHT}{T}$ |
| $N$ | Number of agents |
| $\lambda$ | Arrival rate (calls per interval) |
| $AHT$ | Average Handle Time (seconds) |
| $T$ | Interval duration (seconds) |

### Key Metrics Derived from Erlang-C:

**Average Speed of Answer (ASA):** -> Average time a caller waits before reaching an agent
$$
ASA = \frac{P_w \cdot AHT}{N - A}
$$

**Occupancy Level:** -> percentage of time agents are busy handling calls
$$
\rho = \frac{A}{N}
$$

### Constraints:
- **Stability condition:** $N > A$ (more agents than traffic intensity)
- If $N \leq A$, the queue grows infinitely

In [ ]:
from typing import Optional
import math

def erlang_c(calls, aht, interval, agents, target_time=20, shrinkage=0.0):
    """
    Erlang C metrics for call center with shrinkage adjustment.
    
    Shrinkage accounts for time agents are unavailable due to:
    - Breaks, lunches, meetings
    - Training, coaching
    - Absenteeism, PTO
    - System downtime
    
    Parameters:
        calls (int): Number of calls within the interval 
        aht (float): Average Handle Time in seconds 
        interval (float): Interval duration in seconds (1800 for 30 mins)
        agents (float): Number of agents available to handle calls (N)
        target_time (float): Target answer time in seconds (default 20)
        shrinkage (float): Shrinkage rate as decimal (e.g., 0.30 = 30%)
        
    Returns:
        Dictionary with Pw, ASA, occupancy, service level, and staffing metrics
    """
    # Step 1: Calculate the traffic intensity (Erlangs)
    A = (calls * aht) / interval
    N = agents  # This is the number of agents actually working/available
    
    # Step 2: Calculate Erlang-C probability of wait (Pw)
    # uses int(N) for as we can't loop it in decimals. It needs to be a whole number
    # computes the sum: A^0/0! + A^1/1! + A^2/2! + ... + A^(N-1)/(N-1)!
    N_int = int(N)
    
    sum_terms = 0.0
    term = 1.0  # A^0 / 0! = 1 
    
    for i in range(N_int):
        sum_terms += term
        term *= A / (i + 1) 
        
    # Step 2b: Calculate the probability of wait 
    # Use float N (not int) to keep the function smooth/continuous for optimization
    # Allow our optimizer to use values in decimals (e.g N = 45.3)
    adjustment = N / (N - A)
    numerator = term * adjustment
    denominator = sum_terms + numerator
    
    Pw = numerator / denominator
    
    # Step 3: Calculate Average Speed of Answer (ASA)
    asa = Pw * aht / (N - A)
    
    # Step 4: Calculate Occupancy (utilization)
    occupancy = A / N
    
    # Step 5: Calculate Service Level
    # SL = 1 - Pw * e^(-(N-A) * target_time / AHT) 
    # math.exp(-(N - A) * target_time / aht) 
    # probability that a waiting caller is STILL waiting after target_time seconds
    service_level = 1 - Pw * math.exp(-(N - A) * target_time / aht) 
    
    # Step 6: Calculate total agents needed accounting for shrinkage
    # Formula: Total = Working / (1 - Shrinkage)
    # Example: If we need 45 working agents and shrinkage = 30%:
    #          Total = 45 / (1 - 0.30) = 45 / 0.70 = 65 agents to schedule
    working_agents = N
    total_agents_needed = math.ceil(working_agents / (1 - shrinkage)) if shrinkage < 1 else float('inf')
    
    return {
        'traffic_intensity': A,
        'probability_of_wait': Pw,
        'asa': asa,
        'occupancy': occupancy,
        'service_level': service_level,
        'working_agents': working_agents,
        'total_agents_needed': total_agents_needed,
        'shrinkage': shrinkage,
        'feasible': True
    }

In [2]:
def find_min_agents(calls, aht, interval, target_sl=0.80, target_time=20, max_occupancy=1.0, shrinkage=0.0):
    """
    Find minimum agents needed to meet service level constraints.
    
    Parameters:
        calls: Number of calls in interval
        aht: Average handle time in seconds
        interval: Interval duration in seconds
        target_sl: Target service level (default 0.80 = 80%)
        target_time: Target answer time in seconds (default 20)
        max_occupancy: Maximum allowed occupancy (default 1.0 = 100%)
        shrinkage: Shrinkage rate as decimal (e.g., 0.30 = 30%)
    
    Returns:
        Tuple of (min_working_agents, total_agents_with_shrinkage, metrics_dict)
    """
    A = (calls * aht) / interval
    min_agents = math.ceil(A) + 1  # Must have more agents than traffic
    
    for agents in range(min_agents, min_agents + 100):
        result = erlang_c(calls, aht, interval, agents, target_time, shrinkage)
        
        if result['service_level'] >= target_sl and result['occupancy'] <= max_occupancy:
            return agents, result['total_agents_needed'], result
    
    return None, None, None

In [3]:
if __name__ == "__main__":
    
    print("=" * 70)
    print("ERLANG-C CALCULATOR TEST (WITH SHRINKAGE)")
    print("=" * 70)
    
    # Test Case: Match the website inputs
    calls = 400
    aht = 180  # seconds (3 minutes)
    interval = 1800  # seconds (30 minutes)
    target_time = 20  # seconds
    target_sl = 0.80  # 80%
    shrinkage = 0.30  # 30% shrinkage (breaks, training, absenteeism, etc.)
    
    print(f"Traffic Intensity: {(calls * aht) / interval:.1f} Erlangs")
    print(f"Shrinkage: {shrinkage*100:.0f}%\n")
    
    print("RESULTS BY AGENT COUNT:")
    print("-" * 70)
    print(f"{'Agents':<8} {'Occupancy':<10} {'ASA':<8} {'Pw':<8} {'SL':<10} {'Total w/Shrink':<15}")
    print("-" * 70)
    
    for agents in range(40, 50):
        r = erlang_c(calls, aht, interval, agents, target_time, shrinkage)
        if r['feasible']:
            meets_target = "✓" if r['service_level'] >= target_sl else ""
            print(f"  {agents:<6} {r['occupancy']*100:>6.1f}%    {r['asa']:>5.1f}s   {r['probability_of_wait']*100:>5.1f}%   {r['service_level']*100:>5.1f}%  {meets_target}    {r['total_agents_needed']}")
        else:
            print(f"  {agents:<6} {'100.0%':<10} {'inf':<8} {'100%':<8} {'0.0%':<10}")
    
    print("-" * 70)
    
    # Find optimal
    working, total, metrics = find_min_agents(calls, aht, interval, target_sl, target_time, shrinkage=shrinkage)
    
    print(f"""
ANSWER:
  Working agents needed (on phones): {working}
  Total agents to schedule (with {shrinkage*100:.0f}% shrinkage): {total}
  
  → Service Level: {metrics['service_level']*100:.1f}% (target: {target_sl*100:.0f}%)
  → ASA: {metrics['asa']:.1f} seconds
  → Occupancy: {metrics['occupancy']*100:.1f}%

SHRINKAGE BREAKDOWN:
  If shrinkage = {shrinkage*100:.0f}%, you need to schedule {total} agents
  to have {working} actually available to take calls.
  
  Formula: Total = Working / (1 - Shrinkage)
           {total} ≈ {working} / (1 - {shrinkage}) = {working / (1-shrinkage):.1f}
""")

ERLANG-C CALCULATOR TEST (WITH SHRINKAGE)
Traffic Intensity: 40.0 Erlangs
Shrinkage: 30%

RESULTS BY AGENT COUNT:
----------------------------------------------------------------------
Agents   Occupancy  ASA      Pw       SL         Total w/Shrink 
----------------------------------------------------------------------
  40     100.0%     inf      100%     0.0%      
  41       97.6%    148.1s    82.3%    26.4%      59
  42       95.2%     60.4s    67.1%    46.3%      61
  43       93.0%     32.5s    54.1%    61.2%      62
  44       90.9%     19.4s    43.2%    72.3%      63
  45       88.9%     12.3s    34.1%    80.5%  ✓    65
  46       87.0%      8.0s    26.6%    86.4%  ✓    66
  47       85.1%      5.3s    20.5%    90.6%  ✓    68
  48       83.3%      3.5s    15.6%    93.6%  ✓    69
  49       81.6%      2.3s    11.7%    95.7%  ✓    70
----------------------------------------------------------------------

ANSWER:
  Working agents needed (on phones): 45
  Total agents to schedule (

# ============================================================
# PART 2: OPTIMIZATION WITH SCIPY
# ============================================================
In this section, we use `scipy.optimize` to find the optimal number of agents mathematically

## Scipy Optimization Approach

Every scipy optimization has **3 parts**:

| Component | Description | Our Problem |
|-----------|-------------|-------------|
| **Objective** | What are we minimizing? | Number of agents (cost) |
| **Constraints** | What rules must be followed? | SL ≥ 80%, Occupancy ≤ 85% |
| **Bounds** | What's the valid range? | Agents > Traffic Intensity |

$$
\begin{aligned}
\text{minimize} \quad & N \quad \text{(number of agents)} \\
\text{subject to} \quad & SL(N) \geq 0.80 \\
& \rho(N) \leq 0.85 \\
& N > A \quad \text{(stability)}
\end{aligned}
$$

WHY SLSQP?

It handles constraints by solving a sequence of quadratic problems.We need to minimize our # of agents but you also need to respect bounds and constraints.

What is bounds?
Must have more agents than traffic intensity (N > A), otherwise the queue grows infinitely

In [4]:
from scipy.optimize import minimize
import numpy as np

def find_min_agents_scipy(calls, aht, interval, target_sl=0.80, target_time=20, 
                          max_occupancy=0.85, shrinkage=0.0):
    """
    Find minimum agents using scipy.optimize.minimize (SLSQP method).
    
    OPTIMIZATION SETUP:
    ==================
    1. OBJECTIVE:    minimize N (number of agents)
    2. CONSTRAINTS:  service_level >= target_sl
                     occupancy <= max_occupancy
    3. BOUNDS:       N > A (traffic intensity) for stability
    
    Parameters:
        calls: Number of calls in interval
        aht: Average handle time in seconds
        interval: Interval duration in seconds
        target_sl: Target service level (default 0.80 = 80%)
        target_time: Target answer time in seconds (default 20)
        max_occupancy: Maximum allowed occupancy (default 0.85 = 85%)
        shrinkage: Shrinkage rate as decimal (e.g., 0.30 = 30%)
    
    Returns:
        Dictionary with optimization results
    """
    # Calculate traffic intensity (needed for bounds)
    A = (calls * aht) / interval
    
    # =========================================================================
    # 1. OBJECTIVE FUNCTION: Minimize number of agents
    # =========================================================================
    def objective(N):
        """We want to minimize the number of agents (cost)"""
        return N[0]  # N is passed as array, return scalar
    
    # =========================================================================
    # 2. CONSTRAINTS: Service Level and Occupancy
    # =========================================================================
    # Scipy constraint format: constraint >= 0 means constraint is satisfied
    
    # Constraint 1: service_level >= target_sl
    def constraint_service_level(N):
        if N[0] <= A:
            return -1e10  # Heavily penalize infeasible solutions
        result = erlang_c(calls, aht, interval, N[0], target_time, shrinkage)
        return result['service_level'] - target_sl
    
    # Constraint 2: occupancy <= max_occupancy
    def constraint_occupancy(N):
        if N[0] <= A:
            return -1e10
        result = erlang_c(calls, aht, interval, N[0], target_time, shrinkage)
        return max_occupancy - result['occupancy']
    
    # Package constraints for scipy
    constraints = [
        {'type': 'ineq', 'fun': constraint_service_level},  # SL >= target
        {'type': 'ineq', 'fun': constraint_occupancy}       # occ <= max
    ]
    
    # =========================================================================
    # 3. BOUNDS: Agents must be > traffic intensity for stability
    # =========================================================================
    lower_bound = math.ceil(A) + 1  # Minimum agents for stability
    upper_bound = lower_bound + 200  # Reasonable upper limit
    bounds = [(lower_bound, upper_bound)]
    
    # Initial guess: start at lower bound
    N0 = [lower_bound]
    
    # =========================================================================
    # SOLVE THE OPTIMIZATION
    # =========================================================================
    result = minimize(
        objective,
        x0=N0,
        method='SLSQP',  # Sequential Least Squares Programming (handles constraints)
        bounds=bounds,
        constraints=constraints,
        options={'ftol': 1e-6, 'maxiter': 100}
    )
    
    # Round to integer (can't have fractional agents)
    N_optimal = int(np.ceil(result.x[0]))
    
    # Get final metrics with optimal agents
    final_metrics = erlang_c(calls, aht, interval, N_optimal, target_time, shrinkage)
    
    return {
        'optimal_working_agents': N_optimal,
        'total_agents_needed': final_metrics['total_agents_needed'],
        'service_level': final_metrics['service_level'],
        'occupancy': final_metrics['occupancy'],
        'asa': final_metrics['asa'],
        'probability_of_wait': final_metrics['probability_of_wait'],
        'traffic_intensity': A,
        'shrinkage': shrinkage,
        'optimization_success': result.success,
        'optimization_message': result.message,
        'feasible': final_metrics['feasible']
    }

In [5]:
# ============================================================================
# TEST: Compare Simple Search vs Scipy Optimization
# ============================================================================

# Test parameters
calls = 400
aht = 180  # seconds (3 minutes)
interval = 1800  # seconds (30 minutes)
target_time = 20  # seconds
target_sl = 0.80  # 80%
max_occupancy = 0.85  # 85%
shrinkage = 0.30  # 30%

print("=" * 70)
print("COMPARISON: Simple Search vs Scipy Optimization")
print("=" * 70)
print(f"\nInputs:")
print(f"  Calls: {calls} | AHT: {aht}s | Interval: {interval}s")
print(f"  Target SL: {target_sl*100:.0f}% | Max Occupancy: {max_occupancy*100:.0f}%")
print(f"  Shrinkage: {shrinkage*100:.0f}%")
print(f"  Traffic Intensity: {(calls * aht) / interval:.1f} Erlangs")

# Method 1: Simple Search
working1, total1, metrics1 = find_min_agents(
    calls, aht, interval, target_sl, target_time, max_occupancy, shrinkage
)

# Method 2: Scipy Optimization
result_scipy = find_min_agents_scipy(
    calls, aht, interval, target_sl, target_time, max_occupancy, shrinkage
)

print("\n" + "-" * 70)
print("RESULTS:")
print("-" * 70)
print(f"{'Metric':<25} {'Simple Search':<20} {'Scipy Optimize':<20}")
print("-" * 70)
print(f"{'Working Agents':<25} {working1:<20} {result_scipy['optimal_working_agents']:<20}")
print(f"{'Total (w/ Shrinkage)':<25} {total1:<20} {result_scipy['total_agents_needed']:<20}")
print(f"{'Service Level':<25} {metrics1['service_level']*100:>6.2f}%{'':<12} {result_scipy['service_level']*100:>6.2f}%")
print(f"{'Occupancy':<25} {metrics1['occupancy']*100:>6.2f}%{'':<12} {result_scipy['occupancy']*100:>6.2f}%")
print(f"{'ASA':<25} {metrics1['asa']:>6.2f}s{'':<12} {result_scipy['asa']:>6.2f}s")
print("-" * 70)
print(f"\nScipy Optimization Status: {result_scipy['optimization_message']}")

COMPARISON: Simple Search vs Scipy Optimization

Inputs:
  Calls: 400 | AHT: 180s | Interval: 1800s
  Target SL: 80% | Max Occupancy: 85%
  Shrinkage: 30%
  Traffic Intensity: 40.0 Erlangs

----------------------------------------------------------------------
RESULTS:
----------------------------------------------------------------------
Metric                    Simple Search        Scipy Optimize      
----------------------------------------------------------------------
Working Agents            48                   48                  
Total (w/ Shrinkage)      69                   69                  
Service Level              93.59%              93.59%
Occupancy                  83.33%              83.33%
ASA                         3.51s               3.51s
----------------------------------------------------------------------

Scipy Optimization Status: Optimization terminated successfully


# ============================================================
# PART 3: MULTI-INTERVAL OPTIMIZATION
# ============================================================
Real workforce planning involves scheduling across multiple intervals (e.g., 8:00 AM, 8:30 AM, etc.).
Instead of optimizing each interval in isolation, we can optimize them all at once.

**Why optimize together?**
While we *could* just loop through each interval and optimize them one by one (since they are independent in this simple model), setting it up as a multi-interval problem prepares us for more complex constraints later (like shifts that span multiple intervals).

$$
\begin{aligned}
\text{minimize} \quad & \sum_{t=1}^{T} N_t \\
\text{subject to} \quad & SL_t(N_t) \geq Target \quad \forall t \\
& \rho_t(N_t) \leq MaxOcc \quad \forall t
\end{aligned}
$$

In [6]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import math

def optimize_multi_interval(intervals_data, target_sl=0.80, target_time=20, 
                            max_occupancy=0.85, shrinkage=0.0):
    """
    Optimize staffing for multiple intervals simultaneously using Scipy.
    
    Parameters:
        intervals_data: List of dicts, each containing {'calls': int, 'aht': float, 'interval': float}
        target_sl: Target service level (0.80)
        target_time: Target answer time (20s)
        max_occupancy: Max occupancy (0.85)
        shrinkage: Shrinkage rate (0.30)
        
    Returns:
        DataFrame with results for each interval
    """
    num_intervals = len(intervals_data)
    
    # Pre-calculate traffic intensity for each interval (needed for bounds)
    traffic_intensities = []
    for data in intervals_data:
        A = (data['calls'] * data['aht']) / data['interval']
        traffic_intensities.append(A)
    
    # =========================================================================
    # 1. OBJECTIVE: Minimize TOTAL agents across all intervals
    # =========================================================================
    def objective(N_array):
        return np.sum(N_array)
    
    # =========================================================================
    # 2. CONSTRAINTS: SL and Occupancy for EACH interval
    # =========================================================================
    constraints = []
    
    for i in range(num_intervals):
        # Capture loop variable i using default argument
        def constraint_sl(N_array, idx=i):
            N_val = N_array[idx]
            data = intervals_data[idx]
            A = traffic_intensities[idx]
            
            if N_val <= A: return -1e10
            
            res = erlang_c(data['calls'], data['aht'], data['interval'], N_val, target_time, shrinkage)
            return res['service_level'] - target_sl

        def constraint_occ(N_array, idx=i):
            N_val = N_array[idx]
            data = intervals_data[idx]
            A = traffic_intensities[idx]
            
            if N_val <= A: return -1e10
            
            res = erlang_c(data['calls'], data['aht'], data['interval'], N_val, target_time, shrinkage)
            return max_occupancy - res['occupancy']
            
        constraints.append({'type': 'ineq', 'fun': constraint_sl})
        constraints.append({'type': 'ineq', 'fun': constraint_occ})
    
    # =========================================================================
    # 3. BOUNDS: N > A for each interval
    # =========================================================================
    bounds = []
    initial_guess = []
    
    for A in traffic_intensities:
        lower = math.ceil(A) + 1
        upper = lower + 200
        bounds.append((lower, upper))
        initial_guess.append(lower)
        
    # =========================================================================
    # SOLVE
    # =========================================================================
    result = minimize(
        objective,
        x0=initial_guess,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints,
        options={'ftol': 1e-6, 'maxiter': 100}
    )
    
    # Process results
    optimal_agents = np.ceil(result.x).astype(int)
    
    results = []
    for i, N in enumerate(optimal_agents):
        data = intervals_data[i]
        metrics = erlang_c(data['calls'], data['aht'], data['interval'], N, target_time, shrinkage)
        
        results.append({
            'Interval': i+1,
            'Calls': data['calls'],
            'AHT': data['aht'],
            'Traffic_Intensity': traffic_intensities[i],
            'Optimal_Agents': N,
            'Total_Scheduled': metrics['total_agents_needed'],
            'Service_Level': metrics['service_level'],
            'Occupancy': metrics['occupancy'],
            'ASA': metrics['asa']
        })
        
    return pd.DataFrame(results)

In [7]:
# Test Multi-Interval Optimization
print("=" * 70)
print("MULTI-INTERVAL OPTIMIZATION TEST")
print("=" * 70)

# Create dummy data for a morning shift (8:00 - 10:00)
# Calls ramp up: 100 -> 250 -> 400 -> 350
morning_data = [
    {'calls': 100, 'aht': 180, 'interval': 1800}, # 8:00 - 8:30
    {'calls': 250, 'aht': 180, 'interval': 1800}, # 8:30 - 9:00
    {'calls': 400, 'aht': 180, 'interval': 1800}, # 9:00 - 9:30
    {'calls': 350, 'aht': 180, 'interval': 1800}  # 9:30 - 10:00
]

df_results = optimize_multi_interval(
    morning_data,
    target_sl=0.80,
    target_time=20,
    max_occupancy=0.85,
    shrinkage=0.30
)

# Display nicely
print(df_results.to_string(index=False, float_format=lambda x: "{:.2f}".format(x)))

print("\nTotal Agents Required (Sum of Max):", df_results['Total_Scheduled'].sum())

MULTI-INTERVAL OPTIMIZATION TEST
 Interval  Calls  AHT  Traffic_Intensity  Optimal_Agents  Total_Scheduled  Service_Level  Occupancy  ASA
        1    100  180              10.00              14               20           0.89       0.71 7.84
        2    250  180              25.00              30               43           0.86       0.83 9.00
        3    400  180              40.00              48               69           0.94       0.83 3.51
        4    350  180              35.00              42               61           0.92       0.83 4.66

Total Agents Required (Sum of Max): 193


# ============================================================
# PART 4: VERIFY WITH GIVEN DATASET
# ============================================================
Load the call center forecast data and apply our Erlang-C optimization to calculate staffing requirements.

In [8]:
# Load the call center forecast data
import pandas as pd

# Load daily call data (generated from call-center-forecasts-3-months.ipynb)
daily_df = pd.read_csv("../data/processed/call-center-data-v3-daily(1).csv")

# Load intraday profiles (for distributing daily calls into 30-min intervals)
intraday_df = pd.read_csv("../data/processed/intraday-profiles(1).csv")

print("Daily Data Shape:", daily_df.shape)
print("Intraday Profiles Shape:", intraday_df.shape)
print("\nDaily Data Sample:")
daily_df.head()

Daily Data Shape: (368, 5)
Intraday Profiles Shape: (1344, 4)

Daily Data Sample:


,Date,Incoming Calls,Talk Duration (AVG),Waiting Time (AVG),Product Group
0,2025-03-01,298,184,284,PRODUCT_ABC_DESKTOP_EN_CHAT
1,2025-03-02,32,163,70,PRODUCT_ABC_DESKTOP_EN_CHAT
2,2025-03-03,271,172,120,PRODUCT_ABC_DESKTOP_EN_CHAT
3,2025-03-04,212,163,154,PRODUCT_ABC_DESKTOP_EN_CHAT
4,2025-03-05,247,173,228,PRODUCT_ABC_DESKTOP_EN_CHAT


In [9]:
# ============================================================================
# VERIFICATION 1: Apply Erlang-C to Daily Data (Single Product Group)
# ============================================================================
# For daily data, we'll assume 8 hours of operation (28800 seconds)

WORK_HOURS = 8
INTERVAL_SECONDS = WORK_HOURS * 3600  # 28800 seconds

# Filter for one product group
product_group = "PRODUCT_ABC_DESKTOP_EN_CHAT"
df_product = daily_df[daily_df["Product Group"] == product_group].copy()

print(f"Analyzing: {product_group}")
print(f"Date range: {df_product['Date'].min()} to {df_product['Date'].max()}")
print(f"Total days: {len(df_product)}")

# Apply Erlang-C to each day
results = []
for _, row in df_product.iterrows():
    calls = row['Incoming Calls']
    aht = row['Talk Duration (AVG)']  # Already in seconds
    
    if calls > 0 and aht > 0:
        working, total, metrics = find_min_agents(
            calls=calls,
            aht=aht,
            interval=INTERVAL_SECONDS,
            target_sl=0.80,
            target_time=20,
            max_occupancy=0.85,
            shrinkage=0.30
        )
        
        results.append({
            'Date': row['Date'],
            'Incoming_Calls': calls,
            'AHT_seconds': aht,
            'Actual_Wait_Time': row['Waiting Time (AVG)'],
            'Traffic_Intensity': (calls * aht) / INTERVAL_SECONDS,
            'Working_Agents_Needed': working,
            'Total_Scheduled': total,
            'Predicted_SL': metrics['service_level'] if metrics else None,
            'Predicted_ASA': metrics['asa'] if metrics else None
        })

results_df = pd.DataFrame(results)
results_df['Date'] = pd.to_datetime(results_df['Date'])

print("\n" + "=" * 70)
print("STAFFING RESULTS SUMMARY")
print("=" * 70)
print(f"Average Working Agents Needed: {results_df['Working_Agents_Needed'].mean():.1f}")
print(f"Average Total Scheduled (w/ 30% shrinkage): {results_df['Total_Scheduled'].mean():.1f}")
print(f"Max Agents Needed (Peak Day): {results_df['Total_Scheduled'].max()}")
print(f"Min Agents Needed (Low Day): {results_df['Total_Scheduled'].min()}")

results_df.head(10)

Analyzing: PRODUCT_ABC_DESKTOP_EN_CHAT
Date range: 2025-03-01 to 2025-05-31
Total days: 92

STAFFING RESULTS SUMMARY
Average Working Agents Needed: 2.9
Average Total Scheduled (w/ 30% shrinkage): 4.7
Max Agents Needed (Peak Day): 9
Min Agents Needed (Low Day): 3


,Date,Incoming_Calls,AHT_seconds,Actual_Wait_Time,Traffic_Intensity,Working_Agents_Needed,Total_Scheduled,Predicted_SL,Predicted_ASA
0,2025-03-01,298,184,284,1.903889,4,6,0.879636,13.269373
1,2025-03-02,32,163,70,0.181111,2,3,0.987969,1.347702
2,2025-03-03,271,172,120,1.618472,4,6,0.928829,6.780214
3,2025-03-04,212,163,154,1.199861,3,5,0.886834,12.779817
4,2025-03-05,247,173,228,1.483715,3,5,0.806095,26.362230
5,2025-03-06,194,163,214,1.097986,3,5,0.909620,9.781364
6,2025-03-07,179,164,103,1.019306,3,5,0.925174,7.888291
7,2025-03-08,67,156,119,0.362917,2,3,0.954813,5.311525
8,2025-03-09,27,147,93,0.137813,2,3,0.993104,0.701296
9,2025-03-10,191,158,174,1.047847,3,5,0.920401,8.248356
